In [7]:
import pandas as pd 
import numpy as np 

data = pd.read_pickle('data/pitt_full_interview.pickle')
print(data.head())
print(len(data))

      id    label                                               text
0  138-1  Control  [\n, okay, \n, there, s, a, cookie, jar, on, t...
1  631-0  Control  [\n, the, kids, are, in, the, cookies, \n, the...
2  182-3  Control  [\n, well, Johnny, s, fallin, g, off, the, sto...
3  121-0  Control  [\n, alright, \n, the, boy, is, taking, a, coo...
4  142-3  Control  [\n, mhm, \n, well, the, water, s, running, ov...
1292


In [3]:
anagraphic_data = pd.read_pickle('data/anagraphic_dataframe.pickle')
print(anagraphic_data.head())

   age  education     id  race  sex
0   57         14  001-0     1    1
1   58         14  001-1     1    1
2   59         14  001-2     1    1
3   60         14  001-3     1    1
4   58         16  002-0     1    0


In [9]:
merged_dataframe = pd.merge(data, anagraphic_data, on='id')
print(merged_dataframe)

         id     label                                               text  age  \
0     138-1   Control  [\n, okay, \n, there, s, a, cookie, jar, on, t...   68   
1     631-0   Control  [\n, the, kids, are, in, the, cookies, \n, the...   74   
2     121-0   Control  [\n, alright, \n, the, boy, is, taking, a, coo...   70   
3     142-3   Control  [\n, mhm, \n, well, the, water, s, running, ov...   61   
4     267-2   Control  [\n, clears, throat, mother, is, um, drying, t...   54   
5     105-1   Control  [\n, okay, \n, well, the, mother, is, drying, ...   58   
6     105-0   Control  [\n, alrightie, \n, uh, the, action, I, see, i...   56   
7     140-0   Control  [\n, doesn, t, matter, where, you, start, then...   58   
8     298-1   Control  [\n, okay, \n, the, boy, is, standing, up, try...   66   
9     121-1   Control  [\n, there, s, the, action, \n, there, s, the,...   71   
10    243-0   Control  [\n, okay, the, picture, s, in, a, kitchen, uh...   66   
11    142-0   Control  [\n, 

In [3]:
print(len(data))
print(data.head())

1292
     label                                               text
0  Control  [\n, okay, \n, there, s, a, cookie, jar, on, t...
1  Control  [\n, the, kids, are, in, the, cookies, \n, the...
2  Control  [\n, well, Johnny, s, fallin, g, off, the, sto...
3  Control  [\n, alright, \n, the, boy, is, taking, a, coo...
4  Control  [\n, mhm, \n, well, the, water, s, running, ov...


In [11]:
# noinspection PyUnresolvedReferences
from feature_sets.psycholinguistic import get_psycholinguistic_features
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

sid = SentimentIntensityAnalyzer()

new_dataframe = []
for index, row in tqdm(merged_dataframe.iterrows()): 
    single_sentence_list = []
    
    string = ''
    for token in row.text:
        if token == '\n':
            single_sentence_list.append(string)
            string = ''
        else: 
            string += ' ' + token 
        
    counter = 0 
    comp_sentiment_sum = 0
    for sentence in single_sentence_list:
        ss = sid.polarity_scores(sentence)
        comp_sentiment_sum += ss['compound']
        counter += 1 
    
    if counter != 0:
        average_sentiment = comp_sentiment_sum/counter
    else: 
        average_sentiment = 0 
        
    ## for each interview in the dataset. 
    interview = nltk.pos_tag(row.text, lang='eng')
    
    final_interview = []
    for uttr in interview:
        final_interview.append({'token': uttr[0],'pos':uttr[1]})
    
    dict = get_psycholinguistic_features(final_interview)
    
    dict['average_sentiment'] = average_sentiment
    
    additional_features = []
    
    for  key,value in dict.items():
        additional_features.append(value)
    
    ##Here we take in consideration anagraphic features. 
    
    anagraphic_features = [row.age,row.education,row.race,row.sex]
    
    dict['features'] = additional_features + anagraphic_features
    dict['label'] = row.label 
    dict['text'] = row.text
    dict['mmse']=row.mmse
    
    new_dataframe.append(dict)




0it [00:00, ?it/s]

3it [00:00, 27.18it/s]

9it [00:00, 32.35it/s]

12it [00:00, 31.52it/s]

15it [00:00, 29.41it/s]

...

1087it [00:24, 42.14it/s]

1097it [00:24, 50.92it/s]

1105it [00:25, 57.02it/s]

1115it [00:25, 63.06it/s]

1122it [00:25, 44.35it/s]

In [12]:
### Word correctness 
final_dataframe = pd.DataFrame(new_dataframe)
print(final_dataframe.head())

   average_sentiment                                           features  \
0           0.057656  [314.0, 0.0, 232.5, 1.395, 0.0, 0.05765625, 68...   
1          -0.019138  [471.0, 0.0, 348.75, 2.0925000000000002, 0.0, ...   
2           0.171547  [502.4, 0.0, 372.0, 2.232, 0.0, 0.171547368421...   
3           0.094989  [314.0, 0.0, 232.5, 1.395, 0.0, 0.094988888888...   
4          -0.044973  [0.0, 0.0, 0.0, 0.0, 0.0, -0.04497272727272727...   

   getAoaScore  getConcretenessScore  getFamiliarityScore  \
0       1.3950                   0.0                314.0   
1       2.0925                   0.0                471.0   
2       2.2320                   0.0                502.4   
3       1.3950                   0.0                314.0   
4       0.0000                   0.0                  0.0   

   getImagabilityScore  getSUBTLWordScores    label  \
0               232.50                 0.0  Control   
1               348.75                 0.0  Control   
2               3

In [14]:
import pickle
with open('data/pitt_full_interview_features.pickle', 'wb') as f:
    pickle.dump(final_dataframe, f)